# Manual Relation/Alignment Prototype in Jupyter NB
Purpose: Explore options for inputing relations

In [ ]:
# %pip install ipympl
# %pip install pyqt5

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../.."))  # Repository directory relative to this file.
from MISalign.model.project import Project
from MISalign.model.relation import Relation
from MISalign.model.relation_service import relation_tree
from MISalign.model.image import Image

from PIL import Image as PILImage

from matplotlib import pyplot as plt

In [3]:
from os.path import abspath

In [ ]:
%matplotlib qt5
img=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages01.jpg"))
# display(img)
plt.imshow(img)
plt.ginput(5)
# select up to 5 points using left mouse
# stop collecting points with middle mouse
# undo previous with right click

In [7]:
%matplotlib qt5
img0=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages01.jpg"))
img1=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages02.jpg"))

fig0, ax0 = plt.subplots()
ax0.imshow(img0)
plt.title("Figure 0")

fig1, ax1 = plt.subplots()
ax1.imshow(img1)
plt.title("Figure 1")
plt.show()

In [8]:
fg0=fig0.ginput(3)
fg1=fig1.ginput(3)
print(fg0,fg1)

[(1260.5389610389611, 1098.2012987012988), (848.2012987012988, 1140.409090909091)] [(1268.672932330827, 19.185577580314202), (855.2758031442243, 51.994873547504994)]
